#### Introduction

This is a mini project for SC1003 Introduction to Computation Thinking and Programming in NTU. We are from FCSD-Group 4 consisting of Yan Jie, Samuel, Timothy, Keith and Run Ze.

#### Algorithmic Thinking
The program is a software that aims to handle large csv files containing student details of different tutorial groups and sort them accordingly into even and distributed teams based on their, in order of significance:
1. Average CGPA
2. Faculty
3. Gender

#### Flowchart

#### Pseudocode

#### Libraries and Dependencies

In [ ]:
# Import visualization libraries
import plotly.graph_objects as go

#### Basic File Management and Sorting Functions

In [ ]:
class Student:
    def __init__(self, tgrp, id, faculty, name, gender, cgpa, team = 0):
        self.tutorialGrp = int(tgrp)
        self.studID = id
        self.faculty = faculty
        self.name = name
        self.gender = gender
        self.cgpa = float(cgpa)
        self.team = team
        
    def setTeam(self, team):
        self.team = team
        
    def getTeam(self):
        return self.team
    
    def printStudent(self):
        print(f'{self.tutorialGrp}, {self.studID}, {self.faculty}, {self.name}, {self.gender}, {self.cgpa}, {self.team}')


In [ ]:
# Initialize data
studentList = []
gpaDataList = []
        
with open('records.csv') as records:
    records.__next__() # Skip the header row
    lines = records.read().strip().split('\n')
    
    for line in lines:
        line = line.split(',')
        line[0] = line[0][2:]
        
        student = Student(line[0], line[1], line[2], line[3], line[4], line[5])
        studentList.append(student)

        student.printStudent()

In [ ]:
# Sort dataList function by tutorial group
def sortData(data):
    # Using itemgetter to retrieve the row by the column index, e.g., Tutorial Group is index 0 - First Column
    sortedData = sorted(data, key=lambda Student: Student.tutorialGrp)
    return sortedData

# Sort dataList by tutorial group and CGPA
def sortCGPA(data):    
    # Using itemgetter to retrieve the row by the column index,
    # e.g., Tutorial Group is index 0 - First Column
    #       CGPA is index 5 - Sixth Column
    sortedData = sorted(data, key=lambda Student: (Student.tutorialGrp, Student.cgpa))
    return sortedData

#### Data Analysis with Graphs from Plotly

In [ ]:
# Graph Visualization functions using Plotly
#TODO: Exception Handling
def showCGPAPerGroup():
    cgpa = []
    tgrp = []
    for student in sortCGPA(studentList):
        cgpa.append(student.cgpa)
        tgrp.append(student.tutorialGrp)
        # print(str(student[0]) + ' ' + str(student[5])) # Use for checking sorting

    # Make it into a readable box plot using the python arrays above
    boxTrace = go.Box(
        x = tgrp,
        y = cgpa,
        name = 'CGPA / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(boxTrace)
    fig.update_layout(
        title="CGPA / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="CGPA") # Name the title of the graph
    fig.show() # Show the graph
    
# showCGPAPerGroup()

In [ ]:
#TODO: Exception Handling
def showStudentsPerGroup():
    # Graph on No. of Students Per Tutorial Group
    numPerGrp = {}
    studList = sortData(studentList)

    for student in studList:
        if student.tutorialGrp not in numPerGrp.keys():
            numPerGrp[student.tutorialGrp] = 0

    for updateStudent in studList:
        numPerGrp[updateStudent.tutorialGrp] += 1

    barTrace = go.Bar(
        x = list(numPerGrp.keys()),
        y = list(numPerGrp.values()),
        name = 'Number of Students / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace)
    fig.update_layout(
        title="No. of Students / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph

# showStudentsPerGroup()

In [ ]:
#TODO: Exception Handling
def showFacultyPerGroup(tutorialGrpNum):
    # Graph on Faculty Distribution per Tutorial Group
    facPerGrp = {}
    numPerGrp = {}
    studList = sortData(studentList)

    # Get the number of students per tutorial group
    for student in studList:
        if student.tutorialGrp not in numPerGrp.keys():
            numPerGrp[student.tutorialGrp] = 0
                
        if student.faculty not in facPerGrp.keys():
            facPerGrp[student.faculty] = 0

    for updateStudent in studList:
        numPerGrp[updateStudent.tutorialGrp] += 1

    # Input the tutorial group number to show the distribution for that group
    tgrpToShow = tutorialGrpNum

    # Get the number of students per faculty in a tutorial group
    for i in range(numPerGrp[tgrpToShow] * tgrpToShow - 50,
                numPerGrp[tgrpToShow] * tgrpToShow):
        facPerGrp[studList[i].faculty] += 1 # incremement the facPerGrp dict by referencing the indexed student and their faculty

    barTrace1 = go.Bar(
            x = list(facPerGrp.keys()),
            y = list(facPerGrp.values()),
            name = 'No. of Students / Faculty'
        )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace1)
    fig.update_layout(
        title="No. of Students / Faculty",
        xaxis_title="Faculty",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph
    
# showFacultyPerGroup(1)

In [ ]:
#TODO: Exception Handling
def showGenderPerGroup():
    # Graph on Gender Count per Tutorial Group 
    tgrp = []
    malePerGrp = {}
    femalePerGrp = {}
    studList = sortData(studentList)

    for student in studList:
        if student.tutorialGrp not in tgrp:
            tgrp.append(student.tutorialGrp)
            malePerGrp[student.tutorialGrp] = 0
            femalePerGrp[student.tutorialGrp] = 0
        
    for updateNum in studList:
        if updateNum.gender == 'Male':
            malePerGrp[updateNum.tutorialGrp] += 1
        if updateNum.gender == 'Female':
            femalePerGrp[updateNum.tutorialGrp] += 1

    maleTrace = go.Bar(
        x = tgrp,
        y = list(malePerGrp.values()),
        hovertext='(Tutorial Group, No. Of Males)',
        name = 'Male')

    femaleTrace = go.Bar(
        x = tgrp,
        y = list(femalePerGrp.values()),
        hovertext = '(Tutorial Group, No. Of Females)',
        name = 'Female')

    # Initialize the graph object with go.Figure()
    fig = go.Figure()
    fig.add_trace(maleTrace)
    fig.add_trace(femaleTrace)
    fig.update_layout(
        title = "Gender / Tutorial Group", # Name the title of the graph
        xaxis_title = "Tutorial Group",
        yaxis_title = "No. Of Students")
    fig.show() # Show the graph
    
# showGenderPerGroup()

#### Main Code

In [ ]:
def getGroupMedianCGPA(studentList, groupNum):
    totalGPA = 0.0
    totalStudents = 0
    
    for student in studentList:
        if student.tutorialGrp == int(groupNum):
            totalGPA += student.cgpa
            totalStudents += 1

    return totalGPA / totalStudents

print(getGroupMedianCGPA(studentList, 1))